In [1]:
import numpy as np

> - Four lettersL ICAO code of station identifier
>- Issuance Time (7 letters)
>> - in the from DDHHMMZ
>>  - DD= , HH= MM=
>- Wind (can vary in length)
>>- first 3 digits: wind direction
>>- next 2 digits mean speed
>>- optional G for gus
>>>- followed by 2 or 3 digit maximum speed
>>-next 2-3 digits units
>>>-KT=knots, KMH=km/hr, MPS=m/s
>- if wind direction varies 60 degrees or more, followed by 3 digits V 3 digits
>- prevailing visibility in statue miles: number SM
>>- runway visiual range R number L/R/C / P/M 4 digit value FT/M V / U/D/D
>>>- P=plus, M=minus, V=variability, U=up, D=down, N=no change
>- present weather chart with lots of options
>- sky cover
>>- SKC=sky clear, NSC=no significant clouds, FEW=few, SCT=scattered, BKN=broken, OVC=overcast
>- temperature/dewpoint (2 digit temps/2 digit dp) (M for minus)
>- altimeter setting and indicator

note ZULU time (24 hour time in UTC), use knots, directions in degrees

In [2]:
import metpy #install metpy using conda -- I personally had erros when using pip

In [3]:
from metpy.units import units

In [4]:
from metpy.io import parse_metar_to_dataframe

# Example METAR data
metar_data = 'METAR KATL 112351Z 09004KT 10SM FEW250 13/M02 A3013 RMK AO2 SLP204 T01281017 10139 20128 51006'

# Parse METAR data to a pandas DataFrame
df = parse_metar_to_dataframe(metar_data)

# Display the DataFrame
print(df)


           station_id  latitude  longitude  elevation           date_time  \
station_id                                                                  
KATL             KATL     33.63     -84.45        296 2023-10-11 23:51:00   

            wind_direction  wind_speed  wind_gust  visibility  current_wx1  \
station_id                                                                   
KATL                    90         4.0        NaN    16093.44          NaN   

            ...  air_temperature  dew_point_temperature altimeter  \
station_id  ...                                                     
KATL        ...             13.0                   -2.0     30.13   

            current_wx1_symbol  current_wx2_symbol  current_wx3_symbol  \
station_id                                                               
KATL                         0                   0                   0   

                                           remarks  air_pressure_at_sea_level  \
station_id           

In [5]:
import datetime
from urllib.request import urlopen
import json
from IPython.display import clear_output

# Array containing all the US States 2-Letter codes
states = """AK AL AR AZ CA CO CT DE FL GA HI IA ID IL IN KS KY LA MA MD ME
 MI MN MO MS MT NC ND NE NH NJ NM NV NY OH OK OR PA RI SC SD TN TX UT VA VT
 WA WI WV WY"""
# Array to hold the dictionary correspondent to each station
stations = []
# Looping through states
for state in states.split():
    
    clear_output(wait=True)
    print(f'Loading stations for state {state}')
    
    # Manipulating the uri to the proper network corresponding to the state
    uri = (
        "https://mesonet.agron.iastate.edu/"
        f"geojson/network/{state}_ASOS.geojson"
    )
    # Loading the data from the uri of the state
    data = urlopen(uri)
    # Appending the list of stations gathered, interpreted by json
    stations += json.load(data)["features"]
    
# This bit will clean up the dictionaries that are returned, 
# not important, but you can comment this seciton out and see how the
# original dictionaries look. Not all that pretty.
clear_output(wait=True)
print("Cleaning Up Station Information")
for station in stations:
    clear_output(wait=True)
    print(f"Cleaning Up Station Information for station {station['id']}")
    for key in station['properties'].keys():
        station[key] = station['properties'][key]
    del station['properties']
    station['lon'] = station['geometry']['coordinates'][0]
    station['lat'] = station['geometry']['coordinates'][1]
    del station['geometry']
    del station['type']
    del station['sid']
clear_output(wait=True)
print(f'{len(stations)} stations found!')

2843 stations found!


In [13]:
import numpy as np
import pandas as pd
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
import sys
sys.path.append(root_path)
# Utilities package
#from src.backend import utils

# A custom function that turns strings into ints, floats, dates, or, if all else fails, the original string.
def my_eval(s):
    """
    Converts a string into an integer, float, or date object, depending on the string's content. 
    If the conversion is unsuccessful, returns the original string.

    This function attempts to interpret a string in the following order:
    1. Tries to convert the string to an integer.
    2. If unsuccessful, tries to convert it to a float.
    3. If still unsuccessful, tries to convert it to a date.
    4. If all conversions fail, it returns the original string.

    Parameters:
    - s (str): The string to convert.

    Returns:
    - int, float, datetime.date, or str: The converted value or the original string if all conversions fail.
    """
    try:
        # Try to convert the string to a number (int or float)
        return int(s)
    except ValueError:
        try:
            return float(s)
        except ValueError:
            try:
                # Try to convert the string to a datetime date
                return datetime.datetime.strptime(s, '%Y-%m-%d').date()
            except ValueError:
                # If all conversions fail, return the original string
                return s
            
# Function that will interpret the utf-8 text data recovered and turn it into a dataframe
def text_to_dataframe(data):
    """
    Converts a text block of data into a Pandas DataFrame.

    The function processes a long string of data, where each line represents either a comment, 
    a header (with column names), or a row of data. Lines beginning with '#' are comments and 
    are ignored, as are empty lines. The first line of non-commented text is treated as the header, 
    providing the column names for the DataFrame. Subsequent lines of data are split by commas 
    and added to the respective columns.

    Each entry in the data is processed by the 'my_eval' function to attempt conversion into 
    int, float, or date types, falling back to the original string if necessary.

    Parameters:
    - data (str): The block of text data to convert.

    Returns:
    - DataFrame: A Pandas DataFrame representing the structured data.
    """

    # Dictionary that will become a dataframe
    data_values = {}
    # Looping thorugh lines of data
    for line in data.split('\n'):
        # Ignoring empty lines or commented-out lines
        if not line.startswith('#') and len(line) > 0:
            # The first non-commented line contains the data columns.
            # If there's nothing in the dictionary, it means that we're there, redefine the dictionary
            if len(data_values) == 0:
                # Start a dictionary where the keys are the columns, and entries are empty arrays (which we'll populate)
                data_values = {col:[] for col in line.split(',')}
            # If we're not in the columns line, we're in a line that contains data
            else:
                # Split data and append it to each entry of dictionary
                line_data = line.split(',')
                for i, col in enumerate(data_values.keys()):
                    data_values[col] += [my_eval(line_data[i])]
    return pd.DataFrame(data_values)

# Function to filter stations to ones that have any data in between startts and endts
def filter_stations(stations, startts, endts):
    """
    Filter the list of station dictionaries. Include a station if its operation 
    dates overlap with the provided startts and endts, even partially.

    Parameters:
    - stations (list of dicts): The list of station dictionaries.
    - startts (datetime): The start datetime.
    - endts (datetime): The end datetime.

    Returns:
    - list of dicts: The filtered list of stations.
    """
    filtered_stations = []

    for station in stations:
        # Parse the station's operation start and end dates.
        # It's important to handle cases where these dates are None or empty strings.
        archive_begin_str = station.get('archive_begin', '')
        archive_end_str = station.get('archive_end', '')

        try:
            archive_begin = datetime.datetime.strptime(archive_begin_str, '%Y-%m-%d') if archive_begin_str else None
        except ValueError:
            archive_begin = None

        try:
            # If archive_end is None or an empty string, we assume the station is still operational
            archive_end = datetime.datetime.strptime(archive_end_str, '%Y-%m-%d') if archive_end_str else datetime.datetime.max
        except ValueError:
            archive_end = None

        # Check if the station's operation dates overlap with the query dates.
        # The station is considered valid if its operation dates overlap even partially with the query dates.
        if archive_begin is not None and archive_end is not None:
            if (startts <= archive_end and endts >= archive_begin):
                filtered_stations.append(station)

    return filtered_stations

# Start and end dates for analysis
startts = datetime.datetime(2023, 1, 1)
endts = datetime.datetime(2023, 1, 2)
# Filtering stations to ones that have data in between startts and endtts
stations = filter_stations(stations, startts, endts)
# Base URL to manipulate
service = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=comma&latlon=yes&"
# Adding dates to URL in proper format
service += startts.strftime("year1=%Y&month1=%m&day1=%d&")
service += endts.strftime("year2=%Y&month2=%m&day2=%d&")
# Specifying the station to get data from
station = stations[0]
# These are the data columns that we care about, we'll ignore the rest
columns_of_interest = ['station', 'valid', 'lon', 'lat', 'tmpf',
                    'relh', 'drct', 'sknt', 'p01i', 'skyc1', 'skyc2',
                    'skyc3', 'skyc4', 'skyl1', 'skyl2', 'skyl3', 'skyl4',
                    'wxcodes', 'ice_accretion_1hr', 'metar']
# Building proper uri for the request
uri = f"{service}&station={station['id']}"

clear_output(wait=True)
print(f"Requesting data for station {station['id']} on base-url: {service}")
# Loading raw data for a single station
data = urlopen(uri, timeout=300).read().decode("utf-8")
# Interpreting data to a dataframe
data = text_to_dataframe(data)[columns_of_interest]

# This Section will slightly tweak some aspects of the data to make them nicer and more human-readable

# Interprets the 'valid' entry as a datetime object
data['valid'] = pd.to_datetime(data['valid'])
# Calculates the equivalent timestamp to each date
#data['timestamp'] = data['valid'].apply(utils.to_unix_timestamp)
# Whenever there's missing data, it's filled with an 'M', this code will replace that with NaNs
data = data.replace('M', np.nan)
# This code will create the column `wspd`, which is the measured wind speed in meters per second, instead of knots.
data['wspd'] = data['sknt']*0.514444
# Creates columns of the North and East decomposition of the wind vector
data['wind_E'] = np.sin(data['drct']/180*np.pi)*data['wspd']
data['wind_N'] = np.cos(data['drct']/180*np.pi)*data['wspd']
# Creating a dataframe for the stations found so we can assign an elevation to the data according to the station id.
stations_df = pd.DataFrame(stations)
# Assigning 'elevation' to each entry based on the station id
data = pd.merge(data, stations_df[['id', 'elevation']], left_on='station', right_on='id', how='left')
# Deleting 'id' column that is created as a side-product of the process above
data.drop('id', axis='columns', inplace=True)

# sorting data by date
data.sort_values(by='valid', ignore_index=True, inplace=True)
data

Requesting data for station PADK on base-url: http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=comma&latlon=yes&year1=2023&month1=01&day1=01&year2=2023&month2=01&day2=02&


,station,valid,lon,lat,tmpf,relh,drct,sknt,p01i,skyc1,...,skyl2,skyl3,skyl4,wxcodes,ice_accretion_1hr,metar,wspd,wind_E,wind_N,elevation
0,PADK,2023-01-01 00:00:00,-176.646,51.878,NaN,NaN,360.0,11.0,NaN,SCT,...,2300.0,3500.0,NaN,NaN,NaN,PADK 010000Z AUTO 36011KT SCT016 BKN023 BKN035...,5.658884,-1.386027e-15,5.658884,4.0
1,PADK,2023-01-01 00:05:00,-176.646,51.878,NaN,NaN,360.0,11.0,NaN,SCT,...,2300.0,2900.0,NaN,NaN,NaN,PADK 010005Z AUTO 36011KT SCT016 BKN023 BKN029...,5.658884,-1.386027e-15,5.658884,4.0
2,PADK,2023-01-01 00:10:00,-176.646,51.878,NaN,NaN,350.0,8.0,NaN,SCT,...,2300.0,3500.0,NaN,NaN,NaN,PADK 010010Z AUTO 35008KT SCT016 BKN023 OVC035...,4.115552,-7.146581e-01,4.053028,4.0
3,PADK,2023-01-01 00:15:00,-176.646,51.878,NaN,NaN,330.0,5.0,0.01,FEW,...,2100.0,3500.0,NaN,NaN,NaN,PADK 010015Z AUTO 33005KT FEW014 BKN021 OVC035...,2.572220,-1.286110e+00,2.227608,4.0
4,PADK,2023-01-01 00:20:00,-176.646,51.878,NaN,NaN,330.0,5.0,0.01,FEW,...,2100.0,3500.0,NaN,NaN,NaN,PADK 010020Z AUTO 33005KT FEW012 BKN021 OVC035...,2.572220,-1.286110e+00,2.227608,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,PADK,2023-01-01 23:40:00,-176.646,51.878,NaN,NaN,310.0,13.0,0.01,SCT,...,2600.0,3300.0,NaN,NaN,NaN,PADK 012340Z AUTO 31013KT SCT016 BKN026 OVC033...,6.687772,-5.123131e+00,4.298817,4.0
330,PADK,2023-01-01 23:45:00,-176.646,51.878,NaN,NaN,330.0,16.0,0.01,SCT,...,2800.0,3400.0,NaN,NaN,NaN,PADK 012345Z AUTO 33016KT SCT014 BKN028 OVC034...,8.231104,-4.115552e+00,7.128345,4.0
331,PADK,2023-01-01 23:50:00,-176.646,51.878,NaN,NaN,320.0,14.0,0.01,SCT,...,2400.0,3500.0,NaN,NaN,NaN,PADK 012350Z AUTO 32014KT SCT015 BKN024 OVC035...,7.202216,-4.629495e+00,5.517218,4.0
332,PADK,2023-01-01 23:55:00,-176.646,51.878,NaN,NaN,330.0,14.0,0.01,FEW,...,2400.0,3500.0,NaN,NaN,NaN,PADK 012355Z AUTO 33014KT FEW015 BKN024 OVC035...,7.202216,-3.601108e+00,6.237302,4.0


In [19]:
metardf=data[['metar']].copy()

In [20]:
metardf

,metar
0,PADK 010000Z AUTO 36011KT SCT016 BKN023 BKN035...
1,PADK 010005Z AUTO 36011KT SCT016 BKN023 BKN029...
2,PADK 010010Z AUTO 35008KT SCT016 BKN023 OVC035...
3,PADK 010015Z AUTO 33005KT FEW014 BKN021 OVC035...
4,PADK 010020Z AUTO 33005KT FEW012 BKN021 OVC035...
...,...
329,PADK 012340Z AUTO 31013KT SCT016 BKN026 OVC033...
330,PADK 012345Z AUTO 33016KT SCT014 BKN028 OVC034...
331,PADK 012350Z AUTO 32014KT SCT015 BKN024 OVC035...
332,PADK 012355Z AUTO 33014KT FEW015 BKN024 OVC035...


In [21]:
metardf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   metar   334 non-null    object
dtypes: object(1)
memory usage: 2.7+ KB


In [37]:
metarlist= metardf['metar'].tolist()
metarlist

['PADK 010000Z AUTO 36011KT SCT016 BKN023 BKN035 03/02 A2891 RMK T00300020 MADISHF',
 'PADK 010005Z AUTO 36011KT SCT016 BKN023 BKN029 03/02 A2891 RMK T00300020 MADISHF',
 'PADK 010010Z AUTO 35008KT SCT016 BKN023 OVC035 02/01 A2891 RMK T00200010 MADISHF',
 'PADK 010015Z AUTO 33005KT FEW014 BKN021 OVC035 02/01 A2891 RMK P0001 T00200010 MADISHF',
 'PADK 010020Z AUTO 33005KT FEW012 BKN021 OVC035 02/01 A2891 RMK P0001 T00200010 MADISHF',
 'PADK 010025Z AUTO 36008KT SCT011 BKN021 OVC035 02/01 A2891 RMK P0001 T00200010 MADISHF',
 'PADK 010028Z AUTO 36008KT FEW009 BKN012 OVC028 02/01 A2891 RMK AO2 P0001 PWINO FZRANO $',
 'PADK 010030Z AUTO 01009KT SCT009 BKN015 OVC021 02/01 A2891 RMK P0001 T00200010 MADISHF',
 'PADK 010035Z AUTO 01007KT SCT009 BKN013 OVC021 02/01 A2891 RMK P0001 T00200010 MADISHF',
 'PADK 010040Z AUTO 36005KT BKN009 BKN013 OVC021 02/01 A2891 RMK P0001 T00200010 MADISHF',
 'PADK 010043Z AUTO 36009KT BKN009 OVC021 02/01 A2891 RMK AO2 CIG 007V014 P0001 PWINO FZRANO $',
 'PADK 010

In [38]:
for i in range(334):
    metarlist[i]='METAR ' + metarlist[i]
print(metarlist)

['METAR PADK 010000Z AUTO 36011KT SCT016 BKN023 BKN035 03/02 A2891 RMK T00300020 MADISHF', 'METAR PADK 010005Z AUTO 36011KT SCT016 BKN023 BKN029 03/02 A2891 RMK T00300020 MADISHF', 'METAR PADK 010010Z AUTO 35008KT SCT016 BKN023 OVC035 02/01 A2891 RMK T00200010 MADISHF', 'METAR PADK 010015Z AUTO 33005KT FEW014 BKN021 OVC035 02/01 A2891 RMK P0001 T00200010 MADISHF', 'METAR PADK 010020Z AUTO 33005KT FEW012 BKN021 OVC035 02/01 A2891 RMK P0001 T00200010 MADISHF', 'METAR PADK 010025Z AUTO 36008KT SCT011 BKN021 OVC035 02/01 A2891 RMK P0001 T00200010 MADISHF', 'METAR PADK 010028Z AUTO 36008KT FEW009 BKN012 OVC028 02/01 A2891 RMK AO2 P0001 PWINO FZRANO $', 'METAR PADK 010030Z AUTO 01009KT SCT009 BKN015 OVC021 02/01 A2891 RMK P0001 T00200010 MADISHF', 'METAR PADK 010035Z AUTO 01007KT SCT009 BKN013 OVC021 02/01 A2891 RMK P0001 T00200010 MADISHF', 'METAR PADK 010040Z AUTO 36005KT BKN009 BKN013 OVC021 02/01 A2891 RMK P0001 T00200010 MADISHF', 'METAR PADK 010043Z AUTO 36009KT BKN009 OVC021 02/01 A28

In [45]:
# Example METAR data
#metar_data = 'METAR KATL 112351Z 09004KT 10SM FEW250 13/M02 A3013 RMK AO2 SLP204 T01281017 10139 20128 51006'

df=pd.DataFrame()
# Parse METAR data to a pandas DataFrame
for i in range(334):
    newrow = parse_metar_to_dataframe(metarlist[i])
    df = pd.concat([df, newrow], ignore_index=True)

# Display the DataFrame
print(df)

    station_id  latitude  longitude  elevation           date_time  \
0         PADK     51.88    -176.65          4 2023-10-01 00:00:00   
1         PADK     51.88    -176.65          4 2023-10-01 00:05:00   
2         PADK     51.88    -176.65          4 2023-10-01 00:10:00   
3         PADK     51.88    -176.65          4 2023-10-01 00:15:00   
4         PADK     51.88    -176.65          4 2023-10-01 00:20:00   
..         ...       ...        ...        ...                 ...   
329       PADK     51.88    -176.65          4 2023-10-01 23:40:00   
330       PADK     51.88    -176.65          4 2023-10-01 23:45:00   
331       PADK     51.88    -176.65          4 2023-10-01 23:50:00   
332       PADK     51.88    -176.65          4 2023-10-01 23:55:00   
333       PADK     51.88    -176.65          4 2023-10-01 23:56:00   

     wind_direction  wind_speed  wind_gust  visibility  current_wx1  ...  \
0               360        11.0        NaN         NaN          NaN  ...   
1      

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   station_id                 334 non-null    object        
 1   latitude                   334 non-null    float64       
 2   longitude                  334 non-null    float64       
 3   elevation                  334 non-null    int64         
 4   date_time                  334 non-null    datetime64[ns]
 5   wind_direction             334 non-null    int64         
 6   wind_speed                 334 non-null    float64       
 7   wind_gust                  56 non-null     float64       
 8   visibility                 0 non-null      float64       
 9   current_wx1                0 non-null      float64       
 10  current_wx2                0 non-null      float64       
 11  current_wx3                0 non-null      float64       
 12  low_clou

In [47]:
newdf=pd.concat([metardf, df], axis=1)

In [48]:
newdf

,metar,station_id,latitude,longitude,elevation,date_time,wind_direction,wind_speed,wind_gust,visibility,...,air_temperature,dew_point_temperature,altimeter,current_wx1_symbol,current_wx2_symbol,current_wx3_symbol,remarks,air_pressure_at_sea_level,eastward_wind,northward_wind
0,PADK 010000Z AUTO 36011KT SCT016 BKN023 BKN035...,PADK,51.88,-176.65,4,2023-10-01 00:00:00,360,11.0,NaN,NaN,...,3.0,2.0,28.91,0,0,0,T00300020 MADISHF,979.32,2.694223e-15,-11.000000
1,PADK 010005Z AUTO 36011KT SCT016 BKN023 BKN029...,PADK,51.88,-176.65,4,2023-10-01 00:05:00,360,11.0,NaN,NaN,...,3.0,2.0,28.91,0,0,0,T00300020 MADISHF,979.32,2.694223e-15,-11.000000
2,PADK 010010Z AUTO 35008KT SCT016 BKN023 OVC035...,PADK,51.88,-176.65,4,2023-10-01 00:10:00,350,8.0,NaN,NaN,...,2.0,1.0,28.91,0,0,0,T00200010 MADISHF,979.32,1.389185e+00,-7.878462
3,PADK 010015Z AUTO 33005KT FEW014 BKN021 OVC035...,PADK,51.88,-176.65,4,2023-10-01 00:15:00,330,5.0,NaN,NaN,...,2.0,1.0,28.91,0,0,0,P0001 T00200010 MADISHF,979.32,2.500000e+00,-4.330127
4,PADK 010020Z AUTO 33005KT FEW012 BKN021 OVC035...,PADK,51.88,-176.65,4,2023-10-01 00:20:00,330,5.0,NaN,NaN,...,2.0,1.0,28.91,0,0,0,P0001 T00200010 MADISHF,979.32,2.500000e+00,-4.330127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,PADK 012340Z AUTO 31013KT SCT016 BKN026 OVC033...,PADK,51.88,-176.65,4,2023-10-01 23:40:00,310,13.0,NaN,NaN,...,2.0,1.0,29.25,0,0,0,P0001 T00200010 MADISHF,990.84,9.958578e+00,-8.356239
330,PADK 012345Z AUTO 33016KT SCT014 BKN028 OVC034...,PADK,51.88,-176.65,4,2023-10-01 23:45:00,330,16.0,NaN,NaN,...,2.0,1.0,29.25,0,0,0,P0001 T00200010 MADISHF,990.84,8.000000e+00,-13.856406
331,PADK 012350Z AUTO 32014KT SCT015 BKN024 OVC035...,PADK,51.88,-176.65,4,2023-10-01 23:50:00,320,14.0,NaN,NaN,...,2.0,1.0,29.25,0,0,0,P0001 T00200010 MADISHF,990.84,8.999027e+00,-10.724622
332,PADK 012355Z AUTO 33014KT FEW015 BKN024 OVC035...,PADK,51.88,-176.65,4,2023-10-01 23:55:00,330,14.0,NaN,NaN,...,3.0,2.0,29.25,0,0,0,P0001 T00300020 MADISHF,990.84,7.000000e+00,-12.124356
